This docuement has references enabled through Calico document, to download:

    sudo ipython install-nbextension https://bitbucket.org/ipre/calico/downloads/calico-document-tools-1.0.zip

Then run the cell below

for more information, see
http://jupyter.cs.brynmawr.edu/hub/dblank/public/Examples/Calico%20Document%20Tools%20and%20Bibtex.ipynb

In [1]:
%%javascript
IPython.load_extensions('calico-document-tools');

<IPython.core.display.Javascript object>

#Review of scaling for free subduction

This notebook describes a couple of different ways of scaling the athermal Stokes flow system. In particular, how dimensionless model time can be related to dimensional 'earth' time (length and mass being more trivial).

First, I describe the way sclaing that I have used in recent, unpublished models

Second, the way that Louis handled scaling in <a name="ref-1"/>[(Moresi, Betts, Miller and Cayley, 2014)](#cite-moresi2014dynamics),

Finally, I summarise the paramters used in <a name="ref-2"/>[(OzBench et al., 2008)](#cite-ozbench2008model), with a view to replicating this model in Underworld's New Interface...

#Background

A typical scaling process for __thermal__-Stokes problems results from non-dimensionalisation of the system (or equivalently dimensional analysis.) In this case, the Stokes system (conservation of mass, momentum and energy) is shown to depend on the non-dimensional group of parameters called the Rayleigh number. In modelling a fluid system, one typically chooses a similar Rayleigh number (while individual dimensional paramters that enter the Rayleigh number may be altered any order of magnitude).

The time scale for this 'scaled' system is typically the conductive timescale:

\begin{equation}t_s = L/\kappa\end{equation}

To return the real or 'earth' time , the model time is multiplied by the time scale

\begin{equation}t_e = t_m \times t_s \end{equation}

#Dan's approach for scaling athermal system


For a purely momentum-mass system, the only dimensional paramters that have time dimensions are gravity and viscosity. 

Often, the scaling is derived from the so-called stokes velocity, which derives from the equilibrium velocity for a  buoyant object in the (low-Renolds number) fluid. The velocity (up to a constant) is:

\begin{equation}v_m = \frac{\triangle \rho_m g L_m^2}{ \eta_m}\end{equation}

Where $L_m$ is the representative length scale of the buoyant object, typically the radius of the sphere. As long as the  geometric, kinematic, and dynamic constaints are fulfilled,(e.g. Reynolds numbers $<< 1$), we can compare two systems as follows (using subscripts m and e for model and earth):

\begin{equation}\frac{1}{v_m}\frac{\triangle \rho_m g_m L_m^2}{\eta_m} = \frac{1}{v_e}\frac{\triangle \rho_e g_e L_e^2}{\eta_e} \end{equation}

or 

\begin{equation}v_e =  v_m \frac{   \eta_m \triangle \rho_e g_e L_e^2}{\eta_e \triangle \rho_m g_m L_m^2} \end{equation}

The model is assumed to be non-dimensional, as in the computational world, which simply involves the manipulation of pure numbers. The ratio of $_e/_m$ parameters in the equation are the dimensional scaling factors chosen to normalise the values in our model. Model times can be compared to earth times by:

\begin{equation}t_e =  t_m \frac{\eta_e \triangle \rho_m g_m L_m}{  \eta_m \triangle \rho_e g_e L_e}\end{equation}

or 

\begin{equation}t_e = t_m \frac{\eta_s}{ \rho_s g_s L_s}\end{equation}


This scaling approach is also discussed in <a name="ref-3"/>[(Pysklywec and Cruden, 2004)](#cite-Pysklywec_2004),<a name="ref-4"/>[(Wilson, Spiegelman and vanKeken, 2013)](#cite-wilson2013terraferma),  and has been used extensively by the Underworld community, e.g. <a name="ref-5"/>[(Schellart and Moresi, 2013)](#cite-Schellart_2013)


I typically choose the following values for the 'scale factors':

* $\rho_s$ = 30 kg/ms
* $L_s$ = 110000 m
* $g_s$ = 10 m
* $\eta_s$ = 1.0e20

Meaning that, in dimensionless model units, 

* $g_m$, $\eta_{m(um)}$ ~ 1
* $L_{m(um)}$ ~ 6
* $\Delta \rho_m$ ~ 10

##Moresi 2014

The two cells below contain the working-out of a different scaling system the models in <a name="ref-6"/>[(Moresi, Betts, Miller and Cayley, 2014)](#cite-moresi2014dynamics)(taken from supplementary data). This is a quasi-thermal approach where the timescale is ultimately related to the representative rock thermal diffusivity, even though the energy equation is not being solved. 

In [8]:
import math
# Age in Myr:                              timeMyrs
# Oceanic crust thickness in km:           crustThickness
# Cohesion in MPa:                         cohesion
# Friction coefficient:                    frictionCoeff
# Plume head top / base (km from surface): plumeTop / plumeBase               

timeMyrs = 80.0
crustThickness = 20
cohesion = 12.5
frictionCoeff = 0.066667

plumeTop = 75   
plumeBase = 75

# "constants" & physical properties

Myrs = 3600*24*365.25*1.0e6     # Convert seconds to Myr
Kms = 1000.0                    # Convert m to km 

# Density in kg/m^3 

basaltDensity   = 2900    # Oceanic crust (cold basalt)
mantleDensity   = 3400    # (cold)  
eclogiteDensity = 3500    # (cold)

crustDensity = basaltDensity    # Oceanic crust choose which phase to use for oceanic crust density calculation

gravAcc = 9.81         # gravitational Acceleration m/s^2

# Thermal properties 

Tsurf    = 0.0         # Surface temperature in degrees C     
Tint     = 1300.0      # Interior temperature in degrees C
Tplume   = 1500.0      # Temperature of the plume head in degrees C

TC2K = 273.0           # Kelvin <--> degrees C
kappa = 1e-6           # Thermal diffusivity m^2/s
alpha = 3.0e-5         # Therml expansivity in /K

# Rheology (From Karato & Wu, 1993, assuming wet dislocation creep)

E0 = 240000                                 # Activation energy (J/mol)
V0 = mantleDensity * gravAcc * 5.0e-6 * 1.0 # Activation Volume (m^3/mol) - then scale with approx lithostatic pressure 
R0 = 8.314                                  # Gas constant

# rheology & temperature profile

def viscosity(T,P):
    """ Arrhenius viscosity assuming absolute temperature T and pressure P """
    return math.exp((E0 + V0*P)/(R0*T))

def hsct(depth,time):
    """ Temperature for cooling half space (with Plume as over-ride) """
    T = Tsurf +(Tint-Tsurf) * math.erf(depth/(2*math.sqrt(time*kappa)))
                                       
    if depth < plumeBase*Kms and depth > plumeTop*Kms:
        T = Tplume
        
    return T


# Viscosities are normalised to Tint at base of the layers
referenceViscosity = viscosity(Tint+TC2K,100*Kms);

# Densities are normalised by delta rho mantle (Tint -> Tsurf)
referenceDensityDifference = mantleDensity * (Tint-Tsurf) * alpha
hotMantleDensity = mantleDensity - referenceDensityDifference
plumeHeadMantleDensity = mantleDensity * (Tplume-Tsurf) * alpha

In [13]:
lengthScale = 1000 * Kms
timeScale = lengthScale ** 2.0 / kappa
viscosityScale = 1.0e20 
temperatureScale = Tint - Tsurf

# Scaling - 2) derived

velocityScale = lengthScale / timeScale
massScale = viscosityScale * lengthScale * timeScale
densityScale = massScale / (lengthScale ** 3.0)
gravityScale = lengthScale / (timeScale ** 2.0)
stressScale = viscosityScale * velocityScale / lengthScale

print 'length scale      {:.2e}'.format(lengthScale)
print 'time scale        {:.2e}'.format(timeScale)
print 'viscosity scale   {:.2e}'.format(viscosityScale)
print 'temperature scale {:.2e}'.format(temperatureScale)

print 'velocity scale    {:.2e}'.format(velocityScale)
print 'mass scale        {:.2e}'.format(massScale)
print 'density scale     {:.2e}'.format(densityScale)
print 'gravity scale     {:.2e}'.format(gravityScale)
print 'stress scale      {:.2e}'.format(stressScale)

gprime = gravAcc / gravityScale
drhoprime = (mantleDensity - hotMantleDensity) / densityScale
buoyancyReScale = gprime * drhoprime

print 'buoyancyReScale      {:.2e}'.format(buoyancyReScale)

print 'Note:  g\' = {} drho\' = {}'.format(gprime, drhoprime) 
print 'For numerical stability we can cancel the density and gravity values as follows:'
print '-  set relative densities to 1 and gravity to {:.2e}'.format(buoyancyReScale)
print '-  reset timeScale (Myr) {:.2e} (10000 -> {:.2f} Myr)'.format(timeScale / (3.15e7 * 1.0e6 * buoyancyReScale), 10000.0 * timeScale / (3.15e7 * 1.0e6 * buoyancyReScale))
print '-  reset velocityScale (cm/yr) {:.2e} (0.0001 -> {:.2f} cm/yr)'.format(velocityScale * 3.15e7 * 100 * buoyancyReScale, 0.0001 * velocityScale * 3.15e7 * 100 * buoyancyReScale)

length scale      1.00e+06
time scale        1.00e+18
viscosity scale   1.00e+20
temperature scale 1.30e+03
velocity scale    1.00e-12
mass scale        1.00e+44
density scale     1.00e+26
gravity scale     1.00e-30
stress scale      1.00e+02
buoyancyReScale      1.30e+07
Note:  g' = 9.81e+30 drho' = 1.326e-24
For numerical stability we can cancel the density and gravity values as follows:
-  set relative densities to 1 and gravity to 1.30e+07
-  reset timeScale (Myr) 2.44e-03 (10000 -> 24.40 Myr)
-  reset velocityScale (cm/yr) 4.10e+04 (0.0001 -> 4.10 cm/yr)


* I never really understood the last two steps, where Louis reset the velocity and timescale. 
* I implemented this scaling to a slab model and got very unrealistic times (scaled times)

##Ozbench model

See .../Underworld/InputFiles/+PublishedResults/OzBenchSubduction.xml
###physical units used (dimensionless)

* Domain = 6 * 6 * 1
* gravity = 1.0
* upper mantle viscosity = 1
* lower mantle viscosity = 100
* upper slab visocosity = 200.0
* core slab viscosity = 2.0e4
* lower slab viscosity = 200
* vonMises upper (cohesion) = 0.06
* vonMises lower (cohesion) = 0.06
* slab density = 1.0
* upper mantle density = 0.0
* lower mantle density = 0.0

###Model setup / numerical parameters
* no. slab layers = 3
* lower mantle = True
* Dim = 3
* elementResI = 96
* elementResJ = 48
* elementResK = 96
* nonLinearMaxIterations =  25
* particlesPerCell = 20


#Final thoughts
As Louis has pointed out to me, the uncertainty in upper mantle / asthenosphere viscosity might be on the order of an 1 order of magnitude. Meanwhile plate velocities (instaneous, as well as long-term-average) are much better constrained. His point is that the upper mantle viscocity should be treated as the unknown, while 'real' (earth) times are simply fixed to representative values, e.g a couple of million years for a portion of the slab to sink through the upper mantle. Nevertheless, the conversion between viscosity and timescales is still non-unique, as the examples above demonstrate.

###Length scales

It is fairly typical in Underworld subduction models to use a vertical model domain length of 1, representing 1000 km in earth. This means that typical old lithsophere is about 0.1 model units wide. 

I chose a length scaling based on the 1 model unit being equal to 1 unit of longitude at the equator (111 km). I like this scaling partly because you doen have to use s many small numbers when dealing the lithsohere (i.e, the lithsohere thickness is ~ 1 unit), an also becaus you the 660 km discontinuity can be enforced at 6 model units beneath the surface. I also think that this scaling convenction might to an easier transition to spherical coordinates. 

###Densities

Should we ever use zero density? As in the Ozbench model, it has been common to set the upper mantle density to zero in Underworld subduction models (and furthermore to use a uniform lithospher density of 1). Numerically, this is fine, but one problem with this is that typcal crust is lighter than typical upper mantle peridotite. So if you want to resolve the crust, an you use this scaling, crustal densities should be negative. Using zero densities for materials seems to result in zero (increase) in lithostatic pressure throughout these domains, which has implications for any pressure dependent rheologies we might want to explore. 

Instead of this, I used an arbitrary density value of say ~2500 kg/m3 to 'normalize' my model densities, given that this is typically lighter than any rock values that might be expected to have a significant effect on subduction. Hence keeping all density values positive. Futher review of this may be needed after more experimentation with 'sticky' air models. 

<!--bibtex

@article{Pysklywec_2004,
  doi = {10.1029/2004gc000748},
  url = {http://dx.doi.org/10.1029/2004gc000748},
  year = {2004},
  month = {oct},
  publisher = {Wiley-Blackwell},
  volume = {5},
  number = {10},
  pages = {n/a--n/a},
  author = {Pysklywec, Russell N and Cruden, Alexander R},
  title = {{Coupled crust-mantle dynamics and intraplate tectonics: Two-dimensional numerical and three-dimensional analogue modeling}},
  journal = {Geochemistry Geophysics, Geosystems},
}

@article{Schellart_2013,
  doi = {10.1002/jgrb.50173},
  url = {http://dx.doi.org/10.1002/jgrb.50173},
  year = {2013},
  month = {jun},
  publisher = {Wiley-Blackwell},
  volume = {118},
  number = {6},
  pages = {3221--3248},
  author = {Schellart, Wouter P and Moresi, Louis},
  title = {{A new driving mechanism for backarc extension and backarc shortening through slab sinking induced toroidal and poloidal mantle flow: Results from dynamic subduction models with an overriding plate}},
  journal = {J. Geophys. Res. Solid Earth},
}


@article{wilson2013terraferma,
  title = {{TerraFERMA: The transparent finite element rapid model assembler for multiphysics problems in earth sciences}},
  author = {Wilson, Cian R and Spiegelman, Marc and van Keken, Peter E},
  journal = {Geochem. Geophys. Geosyst., submitted},
  year = {2013},
}

@article{moresi2014dynamics,
  title={Dynamics of continental accretion},
  author={Moresi, Louus and Betts, Pete G and Miller, Meghan S and Cayley, Ross A},
  journal={Nature},
  volume={508},
  number={7495},
  pages={245--248},
  year={2014},
  publisher={Nature Publishing Group}
}

@article{ozbench2008model,
  title={A model comparison study of large-scale mantle--lithosphere dynamics driven by subduction},
  author={OzBench, Mark and Regenauer-Lieb, Klaus and Stegman, Dave R and Morra, Gabriele and Farrington, Rebecca and Hale, Alina and May, Dave A and Freeman, Justin and Bourgouin, Laurent and M{\"u}hlhaus, Hans and others},
  journal={Physics of the Earth and Planetary Interiors},
  volume={171},
  number={1},
  pages={224--234},
  year={2008},
  publisher={Elsevier}
}


-->

#References

<a name="cite-moresi2014dynamics"/><sup>[^](#ref-1) [^](#ref-6) </sup>Moresi, Louus and Betts, Pete G and Miller, Meghan S and Cayley, Ross A. 2014. _Dynamics of continental accretion_.

<a name="cite-ozbench2008model"/><sup>[^](#ref-2) </sup>OzBench, Mark and Regenauer-Lieb, Klaus and Stegman, Dave R and Morra, Gabriele and Farrington, Rebecca and Hale, Alina and May, Dave A and Freeman, Justin and Bourgouin, Laurent and M&uuml;hlhaus, Hans and others. 2008. _A model comparison study of large-scale mantle--lithosphere dynamics driven by subduction_.

<a name="cite-Pysklywec_2004"/><sup>[^](#ref-3) </sup>Pysklywec, Russell N and Cruden, Alexander R. 2004. _Coupled crust-mantle dynamics and intraplate tectonics: Two-dimensional numerical and three-dimensional analogue modeling_. [URL](http://dx.doi.org/10.1029/2004gc000748)

<a name="cite-wilson2013terraferma"/><sup>[^](#ref-4) </sup>Wilson, Cian R and Spiegelman, Marc and van Keken, Peter E. 2013. _TerraFERMA: The transparent finite element rapid model assembler for multiphysics problems in earth sciences_.

<a name="cite-Schellart_2013"/><sup>[^](#ref-5) </sup>Schellart, Wouter P and Moresi, Louis. 2013. _A new driving mechanism for backarc extension and backarc shortening through slab sinking induced toroidal and poloidal mantle flow: Results from dynamic subduction models with an overriding plate_. [URL](http://dx.doi.org/10.1002/jgrb.50173)

